# Chat with BlastAI model

## Step 1: Installing and importing the libraries

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging)
from itertools import product
import textwrap

## Step 2: Loading the BlastAI model

In [ ]:
try:
  mistral_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "Binura/BlastAI",
                                                       quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_quant_type = "nf4", bnb_4bit_compute_dtype = torch.bfloat16, bnb_4bit_use_double_quant = False),
                                                       device_map={"": 0}, torch_dtype = torch.bfloat16, trust_remote_code = True)

  mistral_model.config.use_cache = False
  mistral_model.config.pretraining_tp = 1

except Exception as e:
    print(f"Error loading Mistral model: {e}")
    raise

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

In [ ]:
try:
  mistral_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "Binura/BlastAI", trust_remote_code = True)
  mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
  mistral_tokenizer.padding_side = "right"

except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

## Step 4: Chatting with the BlastAI model

In [ ]:
def validate_user_prompt(prompt):
    if len(prompt) < 5:
        raise ValueError("User prompt must be at least 5 characters long")
    return prompt

In [ ]:
def print_generated_text(generated_text):
    wrapped_text = textwrap.fill(generated_text, width=175)

    lines = wrapped_text.split('\n')

    for line in lines:
        print(line)

In [ ]:
text_generation_pipeline = pipeline("text-generation", model = mistral_model, tokenizer = mistral_tokenizer, device_map= {"": 0})

In [ ]:
logging.set_verbosity(logging.CRITICAL)
question = "A soil has a dry unit weight of 16.5 kN/m3 and a specific gravity of solids as 2.7. What will be its void ratio?"
question = validate_user_prompt(question)
max_length = 325
model_answer = text_generation_pipeline(
    f"<s>[INST] {question} [/INST]",
    do_sample = True,
    temperature = 0.3,
    max_length = max_length,
    max_new_tokens = max_length - len(question),
    top_k = 45,
    top_p = 0.90,
    num_return_sequences = 1,)

In [ ]:
print_generated_text(model_answer[0]['generated_text'])